<a href="https://colab.research.google.com/github/yukinaga/ai_novel/blob/main/section_4/01_writing_novel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI執筆の実践
AIと人間が協力して小説を執筆します。  
今回もGPT-2のモデルに対してファインチューニングは行いませんが、必要であればSection3を参考にファインチューニングを試してください。  
実行に時間がかかるので、「編集」→「ノートブックの設定」の「ハードウェアアクセラレーター」で「GPU」を選択しましょう。   

## ライブラリのインストール
GPT-2が含まれるライブラリtransformers、形態素解析（≒単語分割）のためのライブラリsentencepieceをインストールします。

In [ ]:
!pip install transformers
!pip install sentencepiece

## GPT-2の設定

今回は、13億のパラメータを持つ`rinna/japanese-gpt-1b`を読み込みます。   
https://huggingface.co/rinna/japanese-gpt-1b

In [ ]:
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt-1b")
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt-1b")
model.cuda()  # GPU対応

## 文章を生成する関数
入力文章から続きの文章を生成する関数を設定します。  


In [ ]:
def getarate_sentences(seed_sentence, min_length, max_length):
    x = tokenizer.encode(seed_sentence, return_tensors="pt", add_special_tokens=False)  # 入力
    x = x.cuda()  # GPU対応
    y = model.generate(x, #　入力
                       min_length=min_length,  # 文章の最小長
                       max_length=max_length,  # 文章の最大長
                       do_sample=True,   # 次の単語を確率で選ぶ
                       top_k=50, # Top-Kサンプリング
                       top_p=0.95,  # Top-pサンプリング
                       temperature=1.2,  # 確率分布の調整
                       num_return_sequences=1,  # 生成する文章の数
                       pad_token_id=tokenizer.pad_token_id,  # パディングのトークンID
                       bos_token_id=tokenizer.bos_token_id,  # テキスト先頭のトークンID
                       eos_token_id=tokenizer.eos_token_id,  # テキスト終端のトークンID
                       )  
    generated_sentences = tokenizer.batch_decode(y, skip_special_tokens=True)  # 特殊トークンをスキップして文章に変換
    return generated_sentences

## シードの設定
文章の書き出しを設定します。  
この文章を、「novel_with_ai.txt」に保存します。

In [ ]:
seed_sentence = "東京の空は今日も曇っていた。" #@param {type:"string"}
with open("novel_with_ai.txt", "w") as f:
    f.write(seed_sentence)  # ファイルの保存

## 小説の執筆
以下の手順で小説を執筆しましょう。  

1. 以下のセルを実行し、シード文章をベースにテキストを生成する  
2. 画面左の「ファイル」から「novel_with_ai.txt」を開き、生成された文章を確認する  
3. 「novel_with_ai.txt」の文章を人の手で修正し、保存して新たなシードとする

1-3を繰り返すことで、AIと協力して小説を執筆することができます。   

In [ ]:
with open("novel_with_ai.txt", "r") as f:
    seed_sentence = f.read()  # ファイルの読み込み

min_length = len(seed_sentence)  # 文章の最小長
max_length = min_length + 50  # 文章の最大長

generated_sentence = getarate_sentences(seed_sentence, min_length, max_length)[0]  # 文章の生成

seed_len = len(seed_sentence.replace("\n", "").replace(" ", ""))  # 改行と半角を除いた長さ
generated_sentence = generated_sentence.replace("\n", "").replace(" ", "")  # 改行と半角を除く
seed_sentence = seed_sentence + generated_sentence[seed_len:]  # シードとの差分を追加
print(seed_sentence)

with open("novel_with_ai.txt", "w") as f:
    f.write(seed_sentence)  # 新たな文章を保存